In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

from skopt.space import Integer, Real, Categorical
from skopt import BayesSearchCV
from scipy.stats import uniform, loguniform

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from sklearn.preprocessing import StandardScaler
import pickle

In [2]:
def metrics(tn, fp, fn, tp, metric = ['accuracy']):
    answers = {}
    
    if 'accuracy' in metric or 'all' in metric:
        answers['accuracy'] = (tp + tn) / (tn + fn + fp + tp)
    if 'sensitivity' in metric  or 'all' in metric:
        answers['sensitivity'] = tp / (tp + fn)
    if 'specificity' in metric  or 'all' in metric:
        answers['specificity'] = tn / (tn + fp)
    if 'f1' in metric or 'all' in metric:
        answers['f1'] = tp / (tp + .5*(fp + fn))

    return answers

In [3]:
df = pd.read_csv('data/explored_cmv&unpop_data')

In [4]:
df.drop(columns = ['Unnamed: 0'], inplace = True)

In [5]:
train = df[df['training_set'] == 1]
test = df[df['training_set'] == 0]

In [6]:
x_list = df.select_dtypes(exclude = np.dtype('O')).columns.tolist()
x_list.remove('post_subreddit')
X = df[x_list]
y = df['post_subreddit']

X = X.astype(np.dtype('float64'))

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

- Dummy

In [7]:
df['post_subreddit'].value_counts(normalize = True)

0    0.563149
1    0.436851
Name: post_subreddit, dtype: float64

- XGBoost

In [23]:
xgb_pipe = Pipeline([
    ('xgb', XGBClassifier(base_score = .563149, random_state = 42, booster = 'gbtree', use_label_encoder = False, verbosity = 0))
])

xgb_params = {
    'xgb__n_estimators': Integer(1, 100),
    'xgb__max_depth': Integer(1, 100),
    'xgb__max_delta_step': Real(0, .5),
    'xgb__min_child_weight': Real(0, 2, prior='uniform'),
    'xgb__learning_rate': Real(0, .2, prior='uniform')
}

xgb_bs = BayesSearchCV(estimator = xgb_pipe,
                     search_spaces = xgb_params,
                     scoring = 'roc_auc',
                     n_iter = 50,
                     n_jobs = 8,
                     cv = 5,
                     verbose = 1,
                     refit = True,
                     optimizer_kwargs = {'base_estimator': 'RF'},
                     random_state=42)

In [24]:
xgb_bs.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:    6.1s remaining:    9.2s
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:    6.3s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:  1.4min remaining:  2.0min
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:  1.4min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:  1.9min remaining:  2.8min
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:  1.9min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:   54.8s remaining:  1.4min
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:   55.8s finished


BayesSearchCV(cv=5,
              estimator=Pipeline(steps=[('xgb',
                                         XGBClassifier(base_score=0.563149,
                                                       booster='gbtree',
                                                       colsample_bylevel=None,
                                                       colsample_bynode=None,
                                                       colsample_bytree=None,
                                                       gamma=None, gpu_id=None,
                                                       importance_type='gain',
                                                       interaction_constraints=None,
                                                       learning_rate=None,
                                                       max_delta_step=None,
                                                       max_depth=None,
                                                       min_child_weight=None,
    

In [ ]:
xgb_bs.best_params_

In [ ]:
xgb_preds = xgb_bs.predict(X_test)

In [ ]:
xgb_tn, xgb_fp, xgb_fn, xgb_tp = confusion_matrix(y_test, xgb_preds).ravel()

In [ ]:
metrics(xgb_tn, xgb_fp, xgb_fn, xgb_tp, metric = 'all')

In [26]:
xgb = xgb_bs.best_estimator_['xgb']

In [27]:
pickle.dump(xgb, open('models/XGBoost_rocauc.sav', 'wb'))

- Bagger

In [28]:
bc_pipe = Pipeline([
    ('bc', BaggingClassifier(random_state = 42))
])

bc_params = {
    'bc__n_estimators': Integer(1, 100),
    'bc__max_samples': Real(0, 1, prior='uniform'),
    'bc__max_features': Real(0, 1, prior='uniform')
}

bc_bs = BayesSearchCV(estimator = bc_pipe,
                     search_spaces = bc_params,
                     n_jobs = 8,
                     scoring = 'roc_auc',
                     n_iter = 50,
                     cv = 5,
                     verbose = 1,
                     refit = True,
                     optimizer_kwargs = {'base_estimator': 'RF'},
                     random_state=42)

In [29]:
bc_bs.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:   27.2s remaining:   40.8s
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:   27.5s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:  2.7min remaining:  4.1min
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:  2.8min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:   44.3s remaining:  1.1min
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:   44.6s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:   25.9s remaining:   38.9s
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:   26.2s finished


BayesSearchCV(cv=5,
              estimator=Pipeline(steps=[('bc',
                                         BaggingClassifier(random_state=42))]),
              n_jobs=8, optimizer_kwargs={'base_estimator': 'RF'},
              random_state=42, scoring='roc_auc',
              search_spaces={'bc__max_features': Real(low=0, high=1, prior='uniform', transform='identity'),
                             'bc__max_samples': Real(low=0, high=1, prior='uniform', transform='identity'),
                             'bc__n_estimators': Integer(low=1, high=100, prior='uniform', transform='identity')},
              verbose=1)

In [12]:
bc_bs.best_params_

OrderedDict([('bc__max_features', 0.6170793585023163),
             ('bc__max_samples', 0.7753643788278826),
             ('bc__n_estimators', 89)])

In [13]:
bc_preds = bc_bs.predict(X_test)

In [14]:
bc_tn, bc_fp, bc_fn, bc_tp = confusion_matrix(y_test, bc_preds).ravel()

In [15]:
metrics(bc_tn, bc_fp, bc_fn, bc_tp, metric = 'all')

{'accuracy': 0.8616071428571429,
 'sensitivity': 0.7932389937106918,
 'specificity': 0.9146341463414634,
 'f1': 0.8335398595621644}

In [30]:
bc = bc_bs.best_estimator_['bc']

In [31]:
pickle.dump(bc, open('models/Bagger_rocauc.sav', 'wb'))

- Logistic Regression

In [63]:
logreg_pipe = Pipeline([
    ('logreg', LogisticRegression(random_state = 42, solver = 'saga'))
])

logreg_params = {
    'logreg__penalty': Categorical(['l1', 'l2', 'elasticnet']),
    'logreg__tol': Real(0, .1, prior='uniform'),
    'logreg__C': Real(0.0001, 100, prior='log-uniform'),
    'logreg__class_weight': Categorical(['balanced', None]),
    'logreg__max_iter': Integer(500, 1000),
    'logreg__l1_ratio': Real(0, 1, prior='uniform')
}

logreg_bs = BayesSearchCV(estimator = logreg_pipe,
                     search_spaces = logreg_params,
                     scoring = 'f1',
                     n_iter = 50,
                     n_jobs = 8,
                     cv = 5,
                     refit = True,
                     optimizer_kwargs = {'base_estimator': 'RF'},
                     random_state=42)

In [64]:
logreg_bs.fit(X_train, y_train)

C:\Users\Avanyali\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:1317: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
C:\Users\Avanyali\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


BayesSearchCV(cv=5,
              estimator=Pipeline(steps=[('logreg',
                                         LogisticRegression(random_state=42,
                                                            solver='saga'))]),
              n_jobs=8, optimizer_kwargs={'base_estimator': 'RF'},
              random_state=42, scoring='f1',
              search_spaces={'logreg__C': Real(low=0.0001, high=100, prior='log-uniform', transform='identity'),
                             'logreg__class_weight': Categorical(categories=('balanced', None), prior=None),
                             'logreg__l1_ratio': Real(low=0, high=1, prior='uniform', transform='identity'),
                             'logreg__max_iter': Integer(low=500, high=1000, prior='uniform', transform='identity'),
                             'logreg__penalty': Categorical(categories=('l1', 'l2', 'elasticnet'), prior=None),
                             'logreg__tol': Real(low=0, high=0.1, prior='uniform', transform='identit

In [ ]:
logreg_bs.best_params_

In [ ]:
logreg_preds = logreg_bs.predict(X_test)

In [ ]:
logreg_tn, logreg_fp, logreg_fn, logreg_tp = confusion_matrix(y_test, logreg_preds).ravel()

In [ ]:
metrics(logreg_tn, logreg_fp, logreg_fn, logreg_tp, metric = 'all')

In [65]:
logreg = logreg_bs.best_estimator_['logreg']

In [66]:
pickle.dump(logreg, open('models/LogisticRegression.sav', 'wb'))

- Decision Tree

In [36]:
dtree_pipe = Pipeline([
    ('dtree', DecisionTreeClassifier(random_state = 42))
])

dtree_params = {
    'dtree__criterion': Categorical(['gini', 'entropy']),
    'dtree__splitter': Categorical(['best', 'random']),
    'dtree__max_depth': Integer(1, 100),
    'dtree__min_samples_split': Real(0, .5),
    'dtree__min_samples_leaf': Real(0, .5),
    'dtree__max_features': Categorical(['auto', 'sqrt', 'log2']),
    'dtree__min_impurity_decrease': Real(0, .2, prior='uniform'),
    'dtree__ccp_alpha': Real(0, .2, prior='uniform')
}

dtree_bs = BayesSearchCV(estimator = dtree_pipe,
                     search_spaces = dtree_params,
                     scoring = 'roc_auc',
                     n_iter = 50,
                     cv = 5,
                     n_jobs = 8,
                     refit = True,
                     optimizer_kwargs = {'base_estimator': 'RF'},
                     random_state=42)

In [37]:
dtree_bs.fit(X_train, y_train)

BayesSearchCV(cv=5,
              estimator=Pipeline(steps=[('dtree',
                                         DecisionTreeClassifier(random_state=42))]),
              n_jobs=8, optimizer_kwargs={'base_estimator': 'RF'},
              random_state=42, scoring='roc_auc',
              search_spaces={'dtree__ccp_alpha': Real(low=0, high=0.2, prior='uniform', transform='identity'),
                             'dtree__criterion': Categorical(categories=('gini', 'entropy'), prior=None),
                             'dtr...
                             'dtree__max_features': Categorical(categories=('auto', 'sqrt', 'log2'), prior=None),
                             'dtree__min_impurity_decrease': Real(low=0, high=0.2, prior='uniform', transform='identity'),
                             'dtree__min_samples_leaf': Real(low=0, high=0.5, prior='uniform', transform='identity'),
                             'dtree__min_samples_split': Real(low=0, high=0.5, prior='uniform', transform='identity'),


In [ ]:
dtree_bs.best_params_

In [ ]:
dtree_preds = dtree_bs.predict(X_test)

In [ ]:
dtree_tn, dtree_fp, dtree_fn, dtree_tp = confusion_matrix(y_test, dtree_preds).ravel()

In [ ]:
metrics(dtree_tn, dtree_fp, dtree_fn, dtree_tp, metric = 'all')

In [39]:
dtree = dtree_bs.best_estimator_['dtree']

In [40]:
pickle.dump(dtree, open('models/DecisionTree_roc_auc.sav', 'wb'))

- Random Forest

In [41]:
rf_pipe = Pipeline([
    ('rf', RandomForestClassifier(random_state = 42))
])

rf_params = {
    'rf__criterion': Categorical(['gini', 'entropy']),
    'rf__n_estimators': Integer(5, 200),
    'rf__min_samples_split': Real(0, .5),
    'rf__min_samples_leaf': Real(0, .5),
    'rf__max_depth': Integer(1, 100),
    'rf__max_features': Categorical(['auto', 'sqrt', 'log2']),
    'rf__min_impurity_decrease': Real(0, .2, prior='uniform'),
    'rf__ccp_alpha': Real(0, .2, prior='uniform'),
    'rf__max_samples': Real(0, 1)
}

rf_bs = BayesSearchCV(estimator = rf_pipe,
                     search_spaces = rf_params,
                     scoring = 'roc_auc',
                     n_iter = 50,
                     cv = 5,
                     n_jobs = 8,
                     refit = True,
                     optimizer_kwargs = {'base_estimator': 'RF'},
                     random_state=42)

In [42]:
rf_bs.fit(X_train, y_train)

BayesSearchCV(cv=5,
              estimator=Pipeline(steps=[('rf',
                                         RandomForestClassifier(random_state=42))]),
              n_jobs=8, optimizer_kwargs={'base_estimator': 'RF'},
              random_state=42, scoring='roc_auc',
              search_spaces={'rf__ccp_alpha': Real(low=0, high=0.2, prior='uniform', transform='identity'),
                             'rf__criterion': Categorical(categories=('gini', 'entropy'), prior=None),
                             'rf__max_dept...
                             'rf__max_samples': Real(low=0, high=1, prior='uniform', transform='identity'),
                             'rf__min_impurity_decrease': Real(low=0, high=0.2, prior='uniform', transform='identity'),
                             'rf__min_samples_leaf': Real(low=0, high=0.5, prior='uniform', transform='identity'),
                             'rf__min_samples_split': Real(low=0, high=0.5, prior='uniform', transform='identity'),
               

In [ ]:
rf_bs.best_params_

In [ ]:
rf_preds = rf_bs.predict(X_test)

In [ ]:
rf_tn, rf_fp, rf_fn, rf_tp = confusion_matrix(y_test, rf_preds).ravel()

In [ ]:
rf_tn, rf_fp, rf_fn, rf_tp

In [ ]:
metrics(rf_tn, rf_fp, rf_fn, rf_tp, metric = 'all')

In [43]:
rf = rf_bs.best_estimator_['rf']

In [44]:
pickle.dump(rf, open('models/RandomForest_roc_auc.sav', 'wb'))

- Extra Trees

In [45]:
et_pipe = Pipeline([
    ('et', ExtraTreesClassifier(random_state = 42))
])

et_params = {
    'et__criterion': Categorical(['gini', 'entropy']),
    'et__n_estimators': Integer(5, 200),
    'et__min_samples_split': Real(0, .5),
    'et__min_samples_leaf': Real(0, .5),
    'et__max_depth': Integer(1, 100),
    'et__max_features': Categorical(['auto', 'sqrt', 'log2']),
    'et__min_impurity_decrease': Real(0, .2, prior='uniform'),
    'et__ccp_alpha': Real(0, .2, prior='uniform'),
    'et__max_samples': Real(0, 1)
}

et_bs = BayesSearchCV(estimator = et_pipe,
                     search_spaces = et_params,
                     scoring = 'roc_auc',
                     n_iter = 50,
                     cv = 5,
                     refit = True,
                     n_jobs = 8,
                     optimizer_kwargs = {'base_estimator': 'RF'},
                     random_state=42)

In [46]:
et_bs.fit(X_train, y_train)

BayesSearchCV(cv=5,
              estimator=Pipeline(steps=[('et',
                                         ExtraTreesClassifier(random_state=42))]),
              n_jobs=8, optimizer_kwargs={'base_estimator': 'RF'},
              random_state=42, scoring='roc_auc',
              search_spaces={'et__ccp_alpha': Real(low=0, high=0.2, prior='uniform', transform='identity'),
                             'et__criterion': Categorical(categories=('gini', 'entropy'), prior=None),
                             'et__max_depth'...
                             'et__max_samples': Real(low=0, high=1, prior='uniform', transform='identity'),
                             'et__min_impurity_decrease': Real(low=0, high=0.2, prior='uniform', transform='identity'),
                             'et__min_samples_leaf': Real(low=0, high=0.5, prior='uniform', transform='identity'),
                             'et__min_samples_split': Real(low=0, high=0.5, prior='uniform', transform='identity'),
               

In [ ]:
et_bs.best_params_

In [ ]:
et_preds = et_bs.predict(X_test)

In [ ]:
et_tn, et_fp, et_fn, et_tp = confusion_matrix(y_test, et_preds).ravel()

In [ ]:
et_tn, et_fp, et_fn, et_tp

In [ ]:
metrics(et_tn, et_fp, et_fn, et_tp, metric = 'all')

In [47]:
et = et_bs.best_estimator_['et']

In [48]:
pickle.dump(et, open('models/ExtraTrees_roc_auc.sav', 'wb'))

- To step 6 ->